In [41]:
import pandas as pd

In [3]:
SEED = 1

In [59]:
train= pd.read_csv("../data/external/Credit_scoring/train.csv")
test = pd.read_csv("../data/external/Credit_scoring/test.csv")

In [60]:
train.dropna(inplace=True)

In [61]:
train.head()

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [62]:
test.head()

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,NaN,0.885519,43,0,0.177513,5700.0,4,0,0,0,0.0
1,2,NaN,0.463295,57,0,0.527237,9141.0,15,0,4,0,2.0
2,3,NaN,0.043275,59,0,0.687648,5083.0,12,0,1,0,2.0
3,4,NaN,0.280308,38,1,0.925961,3200.0,7,0,2,0,0.0
4,5,NaN,1.000000,27,0,0.019917,3865.0,4,0,0,0,1.0


In [63]:
X_train = train.drop(columns=["SeriousDlqin2yrs", "Unnamed: 0"], axis = 1)
y_train = train["SeriousDlqin2yrs"]

In [64]:
X_test = test.drop(columns="SeriousDlqin2yrs", axis = 1)
y_test = pd.read_csv("../data/external/Credit_scoring/test-labels.csv")
test = pd.concat([X_test, y_test], axis=1)
test.dropna(inplace=True)
X_test = test.drop(columns=["Probability","Unnamed: 0", "Id"] , axis = 1)
y_test = test["Probability"]

In [72]:
import xgboost

xgb = xgboost.XGBClassifier(
    random_state=SEED, 
    n_jobs = 8,
    n_estimators = 4,
    max_depth = 5

)
xgb.fit(X_train, y_train)

y_pred = xgb.predict(X_test)

In [82]:
import numpy as np

np.mean(y_pred)

0.00957002457002457

In [84]:
def predict(model, data):
    return model.predict(data)

In [91]:
predict(xgb, X_test)

array([0, 0, 0, ..., 0, 0, 0])

In [93]:
y_test_c = (y_test > 0.5).astype(int)

In [94]:
y_test_c

0         0
1         0
2         0
3         0
4         0
         ..
101495    0
101497    0
101498    0
101499    0
101502    0
Name: Probability, Length: 81400, dtype: int64

In [95]:
import dalex as dx

explainer = dx.Explainer(xgb, X_test, y_test_c, predict_function = predict)

Preparation of a new explainer is initiated

  -> data              : 81400 rows 10 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 81400 values
  -> model_class       : xgboost.sklearn.XGBClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function predict at 0x13878a8e0> will be used
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 0.0, mean = 0.00957, max = 1.0
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -1.0, mean = 0.00714, max = 1.0
  -> model_info        : package xgboost

A new explainer has been created!


In [112]:
explainer.model_performance()

,recall,precision,f1,accuracy,auc
XGBClassifier,0.521324,0.910141,0.662927,0.991143,0.760224


In [97]:
X_test

,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,0.885519,43,0,0.177513,5700.0,4,0,0,0,0.0
1,0.463295,57,0,0.527237,9141.0,15,0,4,0,2.0
2,0.043275,59,0,0.687648,5083.0,12,0,1,0,2.0
3,0.280308,38,1,0.925961,3200.0,7,0,2,0,0.0
4,1.000000,27,0,0.019917,3865.0,4,0,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...
101495,0.718874,35,1,0.308047,4125.0,8,0,0,1,2.0
101497,0.045230,67,0,0.012198,5000.0,4,0,0,0,0.0
101498,0.282653,24,0,0.068522,1400.0,5,0,0,0,0.0
101499,0.922156,36,3,0.934217,7615.0,8,0,2,0,4.0


In [102]:
# comparision for less than 35 yo and over 35 yo

protected_variable = X_test.age.apply(lambda x: "over" if x > 35 else "under")
privileged_group = "over"

fobject = explainer.model_fairness(
    protected=protected_variable,
    privileged=privileged_group
)

In [103]:
fobject.fairness_check()

Bias detected in 1 metric: TPR

Conclusion: your model cannot be called fair because 1 criterion exceeded acceptable limits set by epsilon.
It does not mean that your model is unfair but it cannot be automatically approved based on these metrics.

Ratios of metrics, based on 'over'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
            TPR       ACC       PPV  FPR       STP
under  0.637631  0.987915  0.994512  1.0  1.222222


In [115]:
fobject.plot()


Found NaN's or 0's for models: {'XGBClassifier'}
It is advisable to check 'metric_ratios'


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed